In [1]:
import pandas as pd
final_data = pd.read_csv('Final_data.csv')

In [2]:
X = final_data.drop('loan_status', axis = 1)
y = final_data['loan_status']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [4]:
import inspect
import time
from imblearn.over_sampling import SMOTE

In [5]:
from imblearn.over_sampling import SMOTENC

resampler = SMOTE(
    sampling_strategy='auto',
    random_state=42,
    k_neighbors=5
)
start_time1 = time.time()
X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)
end_time1 = time.time()
execution_time1 = end_time1 - start_time1
print("Waktu fitting:", execution_time1, "detik")

Waktu fitting: 1.8092141151428223 detik


In [6]:
from xgboost import XGBClassifier
from tqdm import tqdm
from sklearn.metrics import classification_report

xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100,
    verbose=True
)

# Specify eval_set for evaluation during training
eval_set = [(X_test, y_test)]
start_time2 = time.time()
# Fit the model with eval_set
xgb.fit(X_resampled, y_resampled, eval_set=eval_set)

end_time2 = time.time()
execution_time2 = end_time2 - start_time2
print("Waktu fitting:", execution_time2, "detik")
# Fitting (training) the model with a progress bar
progress_bar = tqdm(total=xgb.n_estimators)

def update_progress_bar(*args, **kwargs):
    progress_bar.update(1)



# Make predictions
y_pred = xgb.predict(X_test)
y_pred_resampled = xgb.predict(X_resampled)

# Evaluate the model
target_names = ['bad', 'poor', 'good']
print(classification_report(y_resampled, y_pred_resampled, target_names=target_names))
print(classification_report(y_test, y_pred, target_names=target_names))

[21:10:22] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[0]	validation_0-mlogloss:0.97855
[1]	validation_0-mlogloss:0.87381
[2]	validation_0-mlogloss:0.78508
[3]	validation_0-mlogloss:0.71209
[4]	validation_0-mlogloss:0.64581
[5]	validation_0-mlogloss:0.58825
[6]	validation_0-mlogloss:0.53776
[7]	validation_0-mlogloss:0.49335
[8]	validation_0-mlogloss:0.45414
[9]	validation_0-mlogloss:0.41909
[10]	validation_0-mlogloss:0.38798
[11]	validation_0-mlogloss:0.36109
[12]	validation_0-mlogloss:0.33510
[13]	validation_0-mlogloss:0.31281
[14]	validation_0-mlogloss:0.29334
[15]	validation_0-mlogloss:0.27423
[16]	validation_0-mlogloss:0.25709
[17]	validation_0-mlogloss:0.24205
[18]	validation_0-mlogloss:0.22793
[19]	validation_0-mlogloss:0.21490
[20]	validation_0-mlogloss:0.20398
[21]	validation_0-mlogloss:0.19381
[22]	validation_0-mlogloss:0.18291
[23]	val

  0%|          | 0/100 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         bad       1.00      0.99      1.00    104802
        poor       1.00      1.00      1.00    104802
        good       1.00      1.00      1.00    104802

    accuracy                           1.00    314406
   macro avg       1.00      1.00      1.00    314406
weighted avg       1.00      1.00      1.00    314406

              precision    recall  f1-score   support

         bad       1.00      0.98      0.99      6597
        poor       0.98      1.00      0.99      2187
        good       1.00      1.00      1.00     26234

    accuracy                           1.00     35018
   macro avg       0.99      0.99      0.99     35018
weighted avg       1.00      1.00      1.00     35018



In [7]:
feature_importance = xgb.feature_importances_
# smote xgboost
# Buat DataFrame untuk menyimpan nama fitur dan nilai gain
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Gain': feature_importance})

# Urutkan DataFrame berdasarkan nilai Gain secara descending
feature_importance_df = feature_importance_df.sort_values(by='Gain', ascending=False)

# Jumlah fitur teratas yang ingin ditampilkan
top_n = 10  # Ganti sesuai dengan jumlah yang Anda inginkan

# Tampilkan nama fitur dan nilai gain teratas
top_features = feature_importance_df.head(top_n)
print(top_features)

                    Feature      Gain
18                out_prncp  0.294926
23  collection_recovery_fee  0.228853
25          last_pymnt_amnt  0.194820
7                     grade  0.095339
10             mths_issue_d  0.060487
20          total_rec_prncp  0.032712
4               funded_amnt  0.025341
24        mths_last_pymnt_d  0.020590
26  mths_last_credit_pull_d  0.017250
21            total_rec_int  0.012609
